> **NB : A ouvrir dans bamboolib**

> Les fichiers produits sont dans app/bso_publis_scopus/08_reporting/2022-08-29/open_access/consolidation/one_shot_scd/

In [5]:
import pandas as pd
import numpy as np
import bamboolib as bam
import plotly.express as px
import sqlite3
from sqlite3 import Error
import sqlalchemy as sqla
from pivottablejs import pivot_ui
import json
# pip install openpyxl

In [72]:
dbEngine=sqla.create_engine('sqlite:////home/docker/app/bso_publis_scopus/09_db/publications.db')
df = pd.read_sql(f'select * from bso_publis_uniques_20220829',dbEngine)
df_all = pd.read_sql(f'select * from bso_publis_all_by_affiliation_20220829',dbEngine)

## [Demande Marai-Livia] Nb publis par éditeurs

In [11]:
d = pd.pivot_table(df, columns=['year'], index=['publisher_by_doiprefix'], values='doi', aggfunc='size', fill_value=0)
import pandas as pd; import numpy as np
d = d.reset_index()
d.columns = [str(column) for column in d.columns]
d = d.rename(columns={"publisher_by_doiprefix":"Editeur"})
d.to_excel("/home/docker/app/bso_publis_scopus/08_reporting/2022-08-29/open_access/consolidation/one_shot_scd/nb_publis_uca_par_editeur_2016-2021.xlsx")

## [Demande Xavier] Liste publis OA Bronze & Hybrid

In [37]:
import pandas as pd; import numpy as np
# Step: Keep rows where (((oa_status_normalized is one of: Bronze) or (oa_status_normalized is one of: Hybrid)) and (oa_host_type_normalized is one of: Editeur)) or (oa_host_type_normalized is one of: Editeur et archive ouverte)
d = df.loc[((df['oa_status_normalized'].isin(['Bronze'])) | (df['oa_status_normalized'].isin(['Hybrid']))) & ((df['oa_host_type_normalized'].isin(['Editeur'])) | (df['oa_host_type_normalized'].isin(['Editeur et archive ouverte'])))]
# Step: Drop columns
d1 = d.drop(columns=['published_date', 'publisher', 'oa_status', 'journal_is_in_doaj','oa_locations_host_type', 'oa_locations_license', 'oa_locations_version', 'oa_host_domain', 'oa_hostdomain_count', 'doi_prefix', 'bso_classification_classe', 'source', 'bso_classification_en'])
d1.to_excel("/home/docker/app/bso_publis_scopus/08_reporting/2022-08-29/open_access/consolidation/one_shot_scd/listing_publis_uca_hybrid_bronze_2016-2021
            .xlsx")

A ajouter manuellement dans pivottable.html

	 <h3>Périmètre</h3>
	  <p>Publications UCA 2016-2021 accessible en OA hybride ou bronze</p>
	  <h3>Dictionnaire de variables</h3>
		 <ul>
		  <li>source_id : Id Scopus</li>
		  <li>doi : DOI</li>
		  <li>year : année de publication</li>
		  <li>corresponding : auteur de correspondance si UCA (avec labo)</li>
		  <li>all_authors : tous auteurs UCA</li>
		  <li>genre : type de publication</li>
		  <li>title : titre</li>
		  <li>journal_name : titre de revue</li>
		  <li>journal_issn_l : ISSN de lien de la revue</li>
		  <li>journal_is_oa : valeurs 0/1, revue full OA gold (ie indexée dans le DOAJ)</li>
		  <li>is_oa_normalized : Accès ouvert / Accès fermé</li>
		  <li>oa_status_normalized : Accès fermé / Gold / Bronze / Hybrid</li>
		  <li>oa_host_type_normalized : Accès fermé / OA site éditeur / OA sur archive ouverte / OA édieteur et archive ouverte</li>
		  <li>oa_repo_normalized : hébergement Hal / Arxiv / autres</li>
		  <li>publisher_by_doiprefix : éditeur (consolidé)</li>
		  <li>bso_classification_fr : discipline</li>
		</ul>

## [Demande Ghislain] Totaux publis par labos avec proportion dépôts dans HAL

In [73]:
# Step: Extract hal number value from each doi
df["hal_count"] = df["oa_hostdomain_count"].apply(lambda x: x.replace("'", '"'))
df["hal_count"] = df["hal_count"].apply(lambda x: list(json.loads(x).values())[0])

# Step: Right Join with df where doi=doi
df_all = pd.merge(df_all, df[['doi', 'hal_count']], how='left', on=['doi'])

# Step: First groupby for counting doi by aff
df_all_groupbyaff_count = df_all.groupby(['aff_internal_id', 'affiliation_name']).agg(all_doi_count=('doi', 'nunique')).reset_index()

# Step: Second groupby for counting hal by aff where hal_count is not null and != 0
df_hal_groupbyaff_count = df_all.loc[(df_all['hal_count'] != 0.0) & (df_all['hal_count'].notna())].groupby(['aff_internal_id', 'affiliation_name']).agg(hal_doi_count=('doi', 'nunique')).reset_index()

# Step: Left Join with df_hal_groupbyaff_count where aff_internal_id=aff_internal_id
df_hal = pd.merge(df_all_groupbyaff_count, df_hal_groupbyaff_count[['aff_internal_id', 'hal_doi_count']], how='left', on=['aff_internal_id'])

# Step: Change data type of ['all_doi_count', 'hal_doi_count'] to Integer
for column_name in ['all_doi_count', 'hal_doi_count']:
    df_hal[column_name] = df_hal[column_name].astype('Int64')
    
# Step: Calculate percent of hal records in new col
df_hal["hal_doi_percent"] = round(df_hal["hal_doi_count"] / df_hal["all_doi_count"] * 100).astype('Int64')

# Step: arrange table form
df_hal = df_hal.drop(columns=["aff_internal_id"])
df_hal.columns = ['Labo', 'Total DOI', 'Total DOI avec dépôt dans HAL', 'Pourcentage de dépôts HAL']

# Step: save
df_hal.to_excel("/home/docker/app/bso_publis_scopus/08_reporting/2022-08-29/open_access/consolidation/one_shot_scd/nb_publis_avec_depot_hal_2016-2021.xlsx")